In [ ]:
#import libraries needed
import pandas as pd
import google_streetview.api
import google_streetview.helpers
import geopandas as gpd 
import pandas as pd
from shapely import ops
import os
import shapely
from shapely.geometry import Point, MultiPoint, shape, mapping, Polygon, LineString
import numpy as np
import psycopg2
import pandana as pdna
import networkx as nx
import multiprocessing as mp
import matplotlib.pyplot as plt
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import time
from shapely import wkt
from shapely.geometry import *
shapely.speedups.enable()

In [ ]:
# data connection 
con = psycopg2.connect(database="walkability", user="postgres", password=***,    host="***",port="5432")

In [ ]:
# getting data from database
centerline_midpoint = gpd.read_postgis("select * from staging.centreline_midpoint", con, crs={'init': 'epsg:2019'})
intersection = gpd.read_postgis("select * from staging.centreline_intersection", con, crs={'init': 'epsg:2019'})
#points = gpd.read_postgis("select * from staging.centreline_points", con, crs={'init': 'epsg:2019'})
landzone = gpd.read_postgis("select * from public.landuse", con, crs={'init': 'epsg:2019'})
# open data castalogue of Toronto City
#zoning by law: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8fef077c-9a14-e922-0c57-f390cd68b8a0
#centreline: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#e4ec3384-056f-aa59-70f7-9ad7706f31a3
#address points data: https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#f71a13c4-fb51-6116-57b7-1f51a8190585

In [ ]:
centerline_midpoint = centerline_midpoint[['geom', 'longitude', 'latitude']]
intersection = intersection[['geom', 'longitude', 'latitude']]


In [ ]:
df1 = pd.DataFrame(centerline_midpoint, copy=True)
df2 = pd.DataFrame(intersection, copy=True)

In [ ]:
df = df1.append(df2, ignore_index = True)
df.to_csv("./allpois.csv")

In [ ]:
df.tail(5)


In [ ]:
landzone.tail(5)

In [ ]:
landzone.groupby(landzone['area_type'])

In [ ]:
landzone.area_desc.unique()

In [ ]:
landzone.groupby(['area_desc']).count()
plt.clf()
landzone.groupby('area_desc').sum().plot(kind='bar')
plt.show()

In [ ]:
landzone.groupby(['area_desc']).sum().transform(lambda x: x/np.sum(x)*100)

In [ ]:
landzone['zones'] = landzone['area_desc']
landzone.drop(['x', 'y', 'longitude', 'latitude'  ], axis=1)

In [ ]:
# categorizing land use 'Residential Apartments', 'Vacant', 'Commercial', residential Singles', 'Other Residential', 'Institutional',
#       'Open Space', 'Other + NA', 'Other Roads/Lanes/Walkways', 'Utilities + Transportation', 'Unknown', 'Industrial',
#       'Residential Townhouses', 'Commercial Recreation', 'Agricultural'
landzone.zones.replace(['Residential Apartments','Residential Singles', 'Residential Townhouses', 'Other Residential'], ['residential', 'residential', 'residential', 'residential'], inplace=True)
landzone.zones.replace(['Commercial','Commercial Recreation'], ['business', 'business'], inplace=True)
landzone.zones.replace(['Vacant'], ['vacant'], inplace=True)
landzone.zones.replace(['Open Space'], ['open space'], inplace=True)
landzone.zones.replace(['Agricultural'], ['agricultural'], inplace=True)
landzone.zones.replace(['Utilities + Transportation', 'Other Roads/Lanes/Walkways'], ['utilities', 'utilities'], inplace=True)
landzone.zones.replace(['Unknown', 'Other + NA'], ['mixed use', 'mixed use'], inplace=True)
landzone.zones.replace(['Unknown', 'Agricultural'], ['miscellaneous', 'miscellaneous'], inplace=True)

In [ ]:
landzone.groupby(['zones']).sum().transform(lambda x: x/np.sum(x)*100)

In [ ]:
landzone.crs = {'init' :'epsg:4326'}
landzone["geometry"] = landzone.geom
print(landzone.geometry)

In [ ]:
crs = {'init': 'epsg:4326'}
geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
geodf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

In [ ]:
## intersect .Note the lack of NaNs; dropped anything that didn't intersect
intersect = gpd.sjoin(geodf, landzone, how="inner")

In [ ]:
intersect.groupby(['zones']).sum()


In [ ]:
intersect.head(2)

In [ ]:
intersect.groupby(['zones']).sum().transform(lambda x: x/np.sum(x)*100)

In [ ]:
Toronto = pd.DataFrame(intersect, copy=True)
Toronto

In [ ]:
#20% sampling
grouped = Toronto.groupby('zones')
grouped.apply(lambda x: x.sample(frac=0.2))

In [ ]:
grouped.count()

In [ ]:
Toronto.apply(lambda x: x.sample(frac=0.2))

In [ ]:
# generating 25% sampling 
rows = Toronto.sample(frac =.25) 
  
# checking if sample is 0.25  
  
if (0.25*(len(Toronto))== len(rows)): 
    print( "Cool") 
    print(len(Toronto), len(rows)) 
  
# display 
rows 

In [ ]:
rows.zones.value_counts()/rows.zones.count()*100